## IMPORTING ALL DEPENDANCIES I NEED FOR THIS PROJECT

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import statsmodels.graphics.correlation as sgc
from statsmodels.graphics.gofplots import qqplot
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## CONNECTION TO MY DATABASE ON POSTGRES

This is the connection link to my database on postgreSQL, the actual connection function is on the file **db_connect.py**

In [ ]:
# Import necessary packages
import pandas as pd
import sys
sys.path.append('..')  # Go up one level to the root folder
from db_connect import connect_to_db

# Step 1: Connect to the database
conn = connect_to_db()

# Step 2: Create a cursor and run a query
cursor = conn.cursor()
query = "SELECT * FROM airbnbs_nairobi.listing_data_yearly;"
cursor.execute(query)

# Step 3: Fetch results and convert to a DataFrame
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Step 4: Display the data
print("Connection successful! Previewing data:")
display(df.head())

## Data Exploration an attempt at understanding my data

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
msno.matrix(df)

## Unpacking Listing Type

To try understand the dataset and the types of AirBNBs we are working with, I've decided to display the airbnbs type we have in our dataset.

In [ ]:
# listing all unique listing types
df['listing_type'].unique()

In [ ]:
# count of each listing type
listing_type_counts = df['listing_type'].value_counts()
listing_type_counts

A visual representation of listing types we have in our dataset

In [ ]:
# create a bar chart 
plt.figure(figsize=(10, 6))
listing_type_counts.plot(kind='bar', color='steelblue')
plt.title('Count of Listings by Type', fontsize=14, fontweight='bold')
plt.xlabel('Listing Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout
plt.show()

# Display the counts
print(listing_type_counts)

Based on the bar chart, the most common unit across the dataset is *An Entire rental Unit* with 129 listings offering it as a listing type. There are a couple of listing types with only 1.


The most interesting listings were, a bus, a Tiny home and a Treehouse, my curiosity got the best of me and i had to check them out. Since the dataset is accompanied by images, let me get them and showcase them here.

In [ ]:
interesting_listing_types = ['Bus', 'Treehouse', 'Tiny home']
filtered_df = df[df['listing_type'].isin(interesting_listing_types)]
filtered_df[['listing_type','cover_photo_url']]

To display the cover photos of these interesting listings.

In [ ]:
from IPython.display import Image, HTML
# Create HTML to display images in a grid
html_content = "<div style='display: flex; flex-wrap: wrap; gap: 20px;'>"
for idx, row in filtered_df.iterrows():
    html_content += f"""
    <div style='text-align: center;'>
        <h4>{row['listing_type']}</h4>
        <h4>{row['listing_name']}</h4>
        <img src='{row['cover_photo_url']}' style='width: 300px; height: 300px; object-fit: cover;'>
    </div>
    """
html_content += "</div>"
display(HTML(html_content))

### Let's check room type as a category

How many unique <b> room types </b> are in our dataset?

In [ ]:
df['room_type'].unique()

So our whole dataset has 3 types of rooms:
1. Private Rooms
2. Entire Homes
3. Hotel Rooms

In [ ]:
plt.figure(figsize=(10, 8))
room_type_counts = df['room_type'].value_counts()
colors = ['#FF6B6B', '#4ECDC4', "#16E313"]

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return f'{val}'
    return my_autopct

plt.pie(room_type_counts, labels=room_type_counts.index, autopct=make_autopct(room_type_counts), 
        colors=colors, startangle=90, textprops={'fontsize': 12})
plt.title('Distribution of Room Types', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Display the counts
print(room_type_counts)

**Entire home/apartment:**
These listings provide guests with exclusive access to the entire property, including all rooms and amenities. This is the most common room type in the dataset, reflecting a preference for fully private accommodations.

**Private room:**
Private room listings offer guests exclusive use of a lockable bedroom, while common areas such as kitchens, bathrooms, and living spaces are shared with other occupants. This room type is the second most common in the dataset.

**Hotel room:**
Hotel room listings represent traditional hotel-style accommodations within the Airbnb platform. Only one such listing appears in the dataset, indicating minimal representation of this room type.

In [ ]:
df_categorized = df.sort_values(['room_type']).reset_index(drop=True)
df_categorized

so based on our listings, let's check the minimum_nights required for each booking

In [ ]:
# Check minimum nights statistics
print("Minimum Nights Statistics:")
print(df['minimum_nights'].describe())
print("\n")

# Create a histogram to show distribution
plt.figure(figsize=(12, 6))
plt.hist(df['minimum_nights'], bins=50, color='steelblue', edgecolor='black')
plt.title('Distribution of Minimum Nights Required', fontsize=14, fontweight='bold')
plt.xlabel('Minimum Nights', fontsize=12)
plt.ylabel('Number of Listings', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

Distribution of Minimum Nights Required (Histogram)
---------------------------------------------------

*   The distribution is **heavily right-skewed**.
    
*   A large majority of listings require **1–2 nights**.
    
*   There are **few but significant long-tail values** (7, 14, 28, 30 nights).
    

**What this means:**

*   Short stays dominate the platform.
    
*   Long minimum-night requirements are **rare but impactful**, likely representing niche listings (e.g., long-term rentals or hosts avoiding frequent turnover).

In [ ]:
# Chart 2: Minimum nights by room type (box plot)
plt.figure(figsize=(10, 6))
df.boxplot(column='minimum_nights', by='room_type')
plt.title('Minimum Nights Required by Room Type', fontsize=14, fontweight='bold')
plt.xlabel('Room Type', fontsize=12)
plt.ylabel('Minimum Nights', fontsize=12)
plt.suptitle('')
plt.tight_layout()
plt.show()

Minimum Nights Required by Room Type (Box Plot)
-----------------------------------------------

*   **Entire home**
    
    *   Most listings require **1–2 nights** minimum.
        
    *   There are **several extreme outliers** (7, 14, 28 nights), indicating some hosts enforce long stays.
        
    *   This suggests higher variability and stricter policies for some entire-home hosts.
        
*   **Hotel room**
    
    *   Almost all listings require **exactly 1 night**.
        
    *   Very little to no variation.
        
    *   This reflects hotel-like flexibility and suitability for short stays.
        
*   **Private room**
    
    *   Typically requires **1–2 nights**, similar to entire homes.
        
    *   However, there are **more extreme outliers** (up to 30 nights), showing that some private-room hosts strongly prefer long-term guests.
        

**Key takeaway:**
Hotel rooms are the most flexible, while entire homes and private rooms show greater variability and occasional long-stay restrictions.

In [ ]:
# Chart 3: Average minimum nights per room type (bar chart)
plt.figure(figsize=(10, 6))
avg_min_nights = df.groupby('room_type')['minimum_nights'].mean()
avg_min_nights.plot(kind='bar', color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
plt.title('Average Minimum Nights by Room Type', fontsize=14, fontweight='bold')
plt.xlabel('Room Type', fontsize=12)
plt.ylabel('Average Minimum Nights', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

Average Minimum Nights by Room Type (Bar Chart)
-----------------------------------------------

*   **Hotel room:** ~**1 night**
    
*   **Entire home:** ~**2 nights**
    
*   **Private room:** ~**3.5 nights**
    

**Interpretation:**

*   On average, **private rooms have the strictest minimum stay requirements**, likely targeting longer-term or more stable tenants.
    
*   **Entire homes** sit in the middle, balancing short-term and longer-term stays.
    
*   **Hotel rooms** prioritize short-term stays almost exclusively.

In [ ]:
# Chart 3.1 (Revised): Median minimum nights per room type
plt.figure(figsize=(10, 6))
median_min_nights = df.groupby('room_type')['minimum_nights'].median()
median_min_nights.plot(kind='bar', color=['#FF6B6B', '#4ECDC4', '#45B7D1'])

plt.title('Median Minimum Nights by Room Type', fontsize=14, fontweight='bold')
plt.xlabel('Room Type', fontsize=12)
plt.ylabel('Median Minimum Nights', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


Interpretation of the Median Minimum Nights Chart
-------------------------------------------------

*   **Entire home = 1 night**Half of entire-home listings require **1 night or less**, and half require more.This indicates that despite some long-stay outliers, the _typical_ entire-home listing is short-stay friendly.
    
*   **Hotel room = 1 night**This confirms **high flexibility**, consistent with hotel-style accommodation where one-night stays are the norm.
    
*   **Private room ≈ 1.5 nights**This suggests that private rooms **tend to require slightly longer stays** than the other room types.The median being higher reflects a **greater concentration of 2+ night minimums**, even though 1-night stays are still common.

In [ ]:
# Print detailed statistics by room type
print("\n=== DETAILED MINIMUM NIGHTS ANALYSIS ===\n")
print(df.groupby('room_type')['minimum_nights'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('Std Dev', 'std')
]).round(2))

Overall Insight
---------------

*   The platform primarily supports **short-term accommodation**.
    
*   **Room type strongly influences host behavior**:
    
    *   Hotel rooms → short, flexible stays
        
    *   Entire homes → moderate flexibility
        
    *   Private rooms → more long-term oriented
        
*   Outliers indicate **strategic host decisions**, not errors, and should be treated carefully in analysis (e.g., capped or analyzed separately).

## Cancellation Policy

In [ ]:
plt.figure(figsize=(10, 8))
cancellation_policy_count = df['cancellation_policy'].value_counts()
colors = ['#FF6B6B', "#6F0DD0", '#45B7D1', "#EDC914", "#4EED14"]
plt.pie(cancellation_policy_count, labels=cancellation_policy_count.index, autopct='%1.1f%%', 
        colors=colors, startangle=90, textprops={'fontsize': 11})
plt.title("Cancellation Policy Distribution", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Display the counts
print(cancellation_policy_count)

Cancellation Policy Distribution
--------------------------------

The dataset is dominated by **Flexible** and **Moderate** cancellation policies, with **43.7%** and **36.7%** listings respectively. This suggests that most hosts prefer policies that allow guests greater freedom to cancel reservations with minimal penalties.

**Firm** and **Strict** policies are less common, appearing in **11.3%** and **8.0%** listings respectively, indicating a smaller segment of hosts who prioritize booking certainty over flexibility.

Only **one** listing follows a **Limited** cancellation policy, making it negligible in the overall distribution.

Key Insight
-----------

Overall, the distribution reflects a **guest-friendly marketplace**, where flexible cancellation options are more prevalent than restrictive policies, likely aimed at attracting short-term and undecided travelers.

In [ ]:
listing_cancellation_crosstab = pd.crosstab(df['room_type'], df['cancellation_policy'])

plt.figure(figsize=(14,6))
listing_cancellation_crosstab.plot(kind='bar', stacked=True, figsize=(14,6), color=['#FF6B6B', "#6F0DD0", '#45B7D1', "#EDC914", "#4EED14"])
plt.title('Room Type vs Cancellation Policy', fontsize=14, fontweight='bold')
plt.xlabel('Listing Type', fontsize=12)
plt.ylabel('Number of Listing', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cancellation Policy', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Print the cross-tabulation
print("\nListing Type vs Cancellation Policy Breakdown:")
print(listing_cancellation_crosstab)


Listing Type vs. Cancellation Policy
------------------------------------

### Entire Home

Entire home listings exhibit the **widest range of cancellation policies**. The majority fall under **Flexible (107)** and **Moderate (96)** policies, indicating a strong preference for guest-friendly cancellation terms. However, a noticeable number of listings apply **Firm (29)** and **Strict (18)** policies, suggesting that some hosts of entire properties seek greater booking security.

### Private Room

Private room listings are **predominantly flexible**, with **Flexible (23)** being the most common policy, followed by **Moderate (14)**. Fewer listings adopt **Firm (5)** or **Strict (6)** policies, reinforcing the notion that private room hosts generally aim to attract short-term or budget-conscious guests through lenient cancellation terms.

### Hotel Room

Only one hotel-room listing appears in the dataset, and it follows a **Flexible** cancellation policy. Due to the extremely small sample size, no broader conclusions can be drawn for this category.

Key Insight
-----------

Across all room types, **Flexible and Moderate cancellation policies dominate**, highlighting a market structure that prioritizes guest convenience. Entire home listings, however, are more likely than private rooms to impose stricter cancellation policies, likely reflecting higher financial risk and operational costs for hosts.